<a href="https://colab.research.google.com/github/indrap23/PJJDA/blob/main/Association.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Association Rule Mining

In [ ]:
import pandas as pd

# 1. Membuat Data Mentah
# Kita buat daftar belanjaan
dataset_mentah = {
    'TID': [1, 2, 3, 4, 5],
    'Items': [
        'Bread, Milk',
        'Bread, Diaper, Beer, Eggs',
        'Milk, Diaper, Beer, Coke',
        'Bread, Milk, Diaper, Beer',
        'Bread, Milk, Diaper, Coke'
    ]
}

# 2. Menampilkan dalam Bentuk Tabel
df = pd.DataFrame(dataset_mentah)

# Menghilangkan index default pandas
df_tampil = df.set_index('TID')

print("=== Tabel Transaksi Belanja ===")
display(df_tampil)

=== Tabel Transaksi Belanja ===


,Items
TID,
1,"Bread, Milk"
2,"Bread, Diaper, Beer, Eggs"
3,"Milk, Diaper, Beer, Coke"
4,"Bread, Milk, Diaper, Beer"
5,"Bread, Milk, Diaper, Coke"


Istilah Penting:
- Itemset: Kumpulan barang (contoh: {Roti, Susu}).
- Support ($s$): Seberapa populer sebuah kombinasi barang? (Frekuensi kemunculan dibagi total transaksi).
- Confidence ($c$): Seberapa kuat hubungannya? (Jika beli A, berapa % kemungkinan beli B?).
- Association Rule: Implikasi dalam bentuk $X \to Y$

In [ ]:
# MENGHITUNG SUPPORT COUNT & SUPPORT SECARA MANUAL
# Kita akan mencari seberapa populer kombinasi: {Milk, Bread, Diaper}

# 1. Definisikan itemset yang dicari
cari_barang = {'Milk', 'Bread', 'Diaper'}
total_transaksi = len(dataset_mentah['Items'])
support_count = 0

print(f"Mencari kombinasi: {cari_barang}")
print("-" * 30)

# 2. Loop ke setiap transaksi untuk mengecek
for i, keranjang_string in enumerate(dataset_mentah['Items']):
    # Ubah string "Bread, Milk" menjadi set {'Bread', 'Milk'} agar bisa dicek
    # Kita pisahkan berdasarkan koma (', ')
    keranjang_set = set(keranjang_string.split(', '))

    # Cek apakah barang yang dicari ada di dalam keranjang ini?
    # issubset artinya "apakah {Milk,Bread,Diaper} bagian dari keranjang?"
    if cari_barang.issubset(keranjang_set):
        print(f"Transaksi {i+1}: DITEMUKAN ✅ (Isi: {keranjang_set})")
        support_count += 1
    else:
        print(f"Transaksi {i+1}: Tidak ada ❌ (Isi: {keranjang_set})")

# 3. Hasil Perhitungan
print("-" * 30)
print(f"Support Count (σ) : {support_count}")

# Support = Support Count / Total Transaksi
support = support_count / total_transaksi
print(f"Support (s)       : {support} (atau {support*100}%)")

Mencari kombinasi: {'Milk', 'Diaper', 'Bread'}
------------------------------
Transaksi 1: Tidak ada ❌ (Isi: {'Milk', 'Bread'})
Transaksi 2: Tidak ada ❌ (Isi: {'Beer', 'Diaper', 'Bread', 'Eggs'})
Transaksi 3: Tidak ada ❌ (Isi: {'Coke', 'Milk', 'Beer', 'Diaper'})
Transaksi 4: DITEMUKAN ✅ (Isi: {'Milk', 'Beer', 'Diaper', 'Bread'})
Transaksi 5: DITEMUKAN ✅ (Isi: {'Coke', 'Milk', 'Diaper', 'Bread'})
------------------------------
Support Count (σ) : 2
Support (s)       : 0.4 (atau 40.0%)


## Apa itu Association Rule?

Setelah tahu barang mana yang populer, kita ingin mencari polanya.

Association Rule adalah implikasi berbentuk $X \to Y$ (Jika beli X, maka beli Y).

Contoh Aturan: $\{Milk, Diaper\} \to \{Beer\}$

Bagaimana cara tahu aturan ini bagus atau tidak? Kita pakai Confidence ($c$).

Confidence mengukur kepastian: "Dari semua orang yang beli Milk & Diaper, berapa persen yang lanjut beli Beer?"

Rumus:

$$Confidence (X \to Y) = \frac{\sigma(X \cup Y)}{\sigma(X)}$$

In [ ]:
# MENGHITUNG CONFIDENCE SECARA MANUAL
# Aturan: Jika beli {Milk, Diaper} -> maka beli {Beer}

# X (Antecedent/Penyebab): {Milk, Diaper}
# Y (Consequent/Akibat)  : {Beer}
itemset_X = {'Milk', 'Diaper'}
itemset_X_dan_Y = {'Milk', 'Diaper', 'Beer'} # Gabungan keduanya

# 1. Hitung Support Count untuk X (Penyebab saja)
count_X = 0
for keranjang_string in dataset_mentah['Items']:
    keranjang_set = set(keranjang_string.split(', '))
    if itemset_X.issubset(keranjang_set):
        count_X += 1

# 2. Hitung Support Count untuk X dan Y (Penyebab + Akibat)
count_XY = 0
for keranjang_string in dataset_mentah['Items']:
    keranjang_set = set(keranjang_string.split(', '))
    if itemset_X_dan_Y.issubset(keranjang_set):
        count_XY += 1

# 3. Hitung Confidence
# Rumus: Jumlah(X+Y) dibagi Jumlah(X)
confidence = count_XY / count_X

print(f"Jumlah transaksi beli {itemset_X}       : {count_X}")
print(f"Jumlah transaksi beli {itemset_X_dan_Y}: {count_XY}")
print("-" * 30)
print(f"Confidence: {confidence:.2f} (atau {confidence*100:.0f}%)")

Jumlah transaksi beli {'Diaper', 'Milk'}       : 3
Jumlah transaksi beli {'Diaper', 'Milk', 'Beer'}: 2
------------------------------
Confidence: 0.67 (atau 67%)


## Apakah Aturan ini Valid? (Pentingnya Lift)

Kita sudah tahu Confidence-nya 67%. Tapi tunggu dulu... Bagaimana jika ternyata orang yang beli Bir itu memang sangat banyak (misal 90% pengunjung toko beli bir)?

Jika Beer memang barang pasaran, maka aturan {Milk, Diaper} -> {Beer} mungkin hanya kebetulan.


Untuk memastikannya, kita gunakan metrik Lift.
Rumus:
$Lift = \frac{Confidence(X \to Y)}{Support(Y)}$
Cara Baca:
- Lift > 1: Ada hubungan kuat (Positif). Beli X benar-benar memicu pembelian Y.
- Lift = 1: Tidak ada hubungan (Independen/Kebetulan).
- Lift < 1: Hubungan negatif (Beli X malah mengurangi peluang beli Y).


In [ ]:
# MENGHITUNG LIFT SECARA MANUAL
# Aturan: {Milk, Diaper} -> {Beer}

# Kita butuh Support dari 'Beer' sendirian (tanpa peduli item lain)
item_akibat = {'Beer'}
count_beer = 0

for keranjang_string in dataset_mentah['Items']:
    keranjang_set = set(keranjang_string.split(', '))
    if item_akibat.issubset(keranjang_set):
        count_beer += 1

# Hitung Support Beer (Benchmark)
support_beer = count_beer / total_transaksi

# Hitung Lift
# Ingat: variable 'confidence' sudah kita hitung di sel sebelumnya (0.67)
lift = confidence / support_beer

print(f"Confidence Aturan : {confidence:.2f}")
print(f"Support Beer (Benchmark) : {support_beer:.2f}")
print("-" * 30)
print(f"Lift Ratio : {lift:.2f}")

if lift > 1:
    print("✅ KESIMPULAN: Hubungan KUAT & VALID (Bukan kebetulan).")
else:
    print("❌ KESIMPULAN: Hanya kebetulan saja.")

Confidence Aturan : 0.67
Support Beer (Benchmark) : 0.60
------------------------------
Lift Ratio : 1.11
✅ KESIMPULAN: Hubungan KUAT & VALID (Bukan kebetulan).


## Computational Complexity

ada 3 barang di toko berapa banyak aturan (rules) nya?





$$C(n, k) = \binom{n}{k} = \frac{n!}{k!(n-k)!}$$

$n$ = Total jenis barang di toko.

$k$ = Jumlah barang dalam satu paket/itemset.

$!$ = Faktorial


$3! = 3 \times 2 \times 1 = 6$


Berapa banyak kombinasi barang (Itemset) yang mungkin dibeli?

Beli 1 Barang ($k=1$):$$C(3,1) = \frac{3!}{1!(2!)} = 3 \text{ (Kopi, Gula, Teh)}$$

Beli 2 Barang ($k=2$):$$C(3,2) = \frac{3!}{2!(1!)} = 3 \text{ ({Kopi, Gula}, {Kopi, Teh}, {Gula, Teh})}$$

Beli 3 Barang ($k=3$):$$C(3,3) = \frac{3!}{3!(0!)} = 1 \text{ ({Kopi, Gula, Teh})}$$


Total Itemset: $3 + 3 + 1 = \mathbf{7 \text{ Itemset}}$.


(Rumus Cepat: $2^d - 1 \rightarrow 2^3 - 1 = 8 - 1 = 7$

berapa banyak total aturannya?

$$R = \sum_{k=1}^{d-1} \left[ \binom{d}{k} \times \sum_{j=1}^{d-k} \binom{d-k}{j} \right]$$



$$R = 3^3 - 2^{3+1} + 1$$$$R = 27 - 2^4 + 1$$$$R = 27 - 16 + 1$$$$R = \mathbf{12 \text{ Aturan}}$$

Bukti 12 Aturan tersebut:

- {Kopi} $\to$ {Gula}
- {Kopi} $\to$ {Teh}
- {Gula} $\to$ {Kopi}
- {Gula} $\to$ {Teh}
- {Teh} $\to$ {Kopi}
- {Teh} $\to$ {Gula}
- {Kopi, Gula} $\to$ {Teh}
- {Kopi, Teh} $\to$ {Gula}
- {Gula, Teh} $\to$ {Kopi}
- {Kopi} $\to$ {Gula, Teh}
- {Gula} $\to$ {Kopi, Teh}
- {Teh} $\to$ {Kopi, Gula}

bagaimana kalo 6 barang?

$$3^6 - 2^7 + 1 = 729 - 128 + 1 = 602 \text{ aturan.}$$



## Menggunakan Algoritma Apriori

<b>Masalah Cara Manual</b>

Bayangkan jika toko Anda punya 10.000 jenis barang. Jika kita cek satu-satu (Brute Force), kita harus menghitung miliaran kombinasi. Itu mustahil.


Tujuan kita adalah mencari semua aturan yang memenuhi syarat:

Support $\ge$ Minsup (Populer)

Confidence $\ge$ Minconf (Kuat)

Jika kita menggunakan cara biasa (Brute-force), kita harus menghitung miliaran kemungkinan. Ini adalah masalahnya karena komputer tidak sanggup (computationally prohibitive)

Brute-Force: Mengecek semua kombinasi item satu per satu.


Apriori: Menggunakan Prinsip Apriori untuk memangkas (prune) kandidat yang tidak perlu dicek .

$$\forall X, Y : (X \subseteq Y) \Rightarrow s(X) \ge s(Y)$$

"Untuk semua item, jika itemset $X$ merupakan bagian dari itemset $Y$, maka frekuensi kemunculan (support) itemset $X$ pasti lebih besar atau sama dengan itemset $Y$."


Misalkan $X$ = {Nasi Goreng}

Misalkan $Y$ = {Nasi Goreng + Telur Dadar}


Jumlah orang yang beli Nasi Goreng ($s(X)$) PASTI lebih banyak (atau sama) daripada orang yang beli Paket Nasi Goreng + Telur ($s(Y)$).



pruning (pemangkasan)?

Karena kita tahu $s(X) \ge s(Y)$, maka:Jika $s(X)$ sudah kecil (jarang dibeli), maka...Kita TIDAK PERLU MENGHITUNG $s(Y)$.





In [ ]:
import pandas as pd
from mlxtend.preprocessing import TransactionEncoder
from mlxtend.frequent_patterns import apriori

# --- BAGIAN 1: INPUT DATA SESUAI GAMBAR ---
# Data 5 Transaksi persis seperti di slide
dataset = [
    ['Bread', 'Milk'],                          # T1
    ['Beer', 'Bread', 'Diaper', 'Eggs'],        # T2
    ['Beer', 'Coke', 'Diaper', 'Milk'],         # T3
    ['Beer', 'Bread', 'Diaper', 'Milk'],        # T4
    ['Bread', 'Coke', 'Diaper', 'Milk']         # T5
]

# Parameter sesuai slide (Min Support = 3)
min_count = 3
total_transaksi = len(dataset)
min_support = min_count / total_transaksi  # 3 dibagi 5 = 0.6

print(f"Total Transaksi: {total_transaksi}")
print(f"Syarat Minimal (Count): {min_count}")
print(f"Syarat Minimal (Support): {min_support} (60%)")
print("-" * 40)

# --- BAGIAN 2: PROSES APRIORI ---

# 1. Mengubah data text menjadi matrix True/False (One-Hot Encoding)
te = TransactionEncoder()
te_ary = te.fit(dataset).transform(dataset)
df = pd.DataFrame(te_ary, columns=te.columns_)

# 2. Menjalankan Algoritma Apriori
# Algoritma ini otomatis melakukan "Pruning" (membuang yang tidak lolos)
frequent_itemsets = apriori(df, min_support=min_support, use_colnames=True)

# 3. Menambahkan kolom 'Count' agar sesuai dengan tampilan di slide
frequent_itemsets['count'] = frequent_itemsets['support'] * total_transaksi
frequent_itemsets['length'] = frequent_itemsets['itemsets'].apply(lambda x: len(x))

# --- BAGIAN 3: MENAMPILKAN HASIL SESUAI TAHAPAN SLIDE ---

print("\n--- TAHAP 1: 1-Itemsets (Barang Satuan) ---")
# Filter hanya yang 1 barang
step1 = frequent_itemsets[frequent_itemsets['length'] == 1]
if not step1.empty:
    print(step1[['itemsets', 'count']].to_string(index=False))
else:
    print("Tidak ada yang lolos.")
print("\n(Catatan: 'Coke' (2) dan 'Eggs' (1) sudah dibuang otomatis karena < 3)")


print("\n--- TAHAP 2: 2-Itemsets (Pasangan) ---")
# Filter hanya yang 2 barang
step2 = frequent_itemsets[frequent_itemsets['length'] == 2]
if not step2.empty:
    print(step2[['itemsets', 'count']].to_string(index=False))
else:
    print("Tidak ada pasangan yang lolos.")
print("\n(Catatan: Pasangan seperti {Bread, Beer} dibuang karena cuma muncul 2 kali)")


print("\n--- TAHAP 3: 3-Itemsets (Tiga Barang) ---")
# Filter hanya yang 3 barang
step3 = frequent_itemsets[frequent_itemsets['length'] == 3]

if not step3.empty:
    print(step3[['itemsets', 'count']].to_string(index=False))
else:
    print("KOSONG / TIDAK ADA YANG LOLOS.")
    print("Alasan: Sesuai slide, kandidat {Bread, Diaper, Milk} hanya muncul 2 kali.")

print("-" * 40)
print("Kesimpulan: Hasil akhir kode ini SAMA PERSIS dengan kotak kuning di slide.")

/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)
/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)
/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)
/usr/local/lib/python3.12/dist-packag

Total Transaksi: 5
Syarat Minimal (Count): 3
Syarat Minimal (Support): 0.6 (60%)
----------------------------------------

--- TAHAP 1: 1-Itemsets (Barang Satuan) ---
itemsets  count
  (Beer)    3.0
 (Bread)    4.0
(Diaper)    4.0
  (Milk)    4.0

(Catatan: 'Coke' (2) dan 'Eggs' (1) sudah dibuang otomatis karena < 3)

--- TAHAP 2: 2-Itemsets (Pasangan) ---
       itemsets  count
 (Beer, Diaper)    3.0
(Diaper, Bread)    3.0
  (Milk, Bread)    3.0
 (Milk, Diaper)    3.0

(Catatan: Pasangan seperti {Bread, Beer} dibuang karena cuma muncul 2 kali)

--- TAHAP 3: 3-Itemsets (Tiga Barang) ---
KOSONG / TIDAK ADA YANG LOLOS.
Alasan: Sesuai slide, kandidat {Bread, Diaper, Milk} hanya muncul 2 kali.
----------------------------------------
Kesimpulan: Hasil akhir kode ini SAMA PERSIS dengan kotak kuning di slide.


/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)
/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)
/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)
/usr/local/lib/python3.12/dist-packag

## Extensions of Association Analysis to Continuous and Categorical Attributes and Multi-level Rules

Apriori standar (Basic) hanya bekerja pada Binary Data (0/1, Beli/Tidak, True/False) dan menganggap semua barang setara.

Di dunia nyata, data jauh lebih kompleks:
- Atribut Kategorikal: Warna (Merah, Biru), Merek (Honda, Toyota).
- Atribut Kontinu: Harga ($10.50, $500), Usia (25, 60).
- Hierarki: Produk punya level (Susu $\rightarrow$ Produk Susu $\rightarrow$ Makanan).
- Urutan (Sequence): Waktu pembelian berpengaruh (Beli HP dulu, baru beli Casing).


**Handling Categorical Attributes (Menangani Kategori)**

Data transaksi seringkali punya atribut seperti Brand: Apple atau Color: Red. Kita tidak bisa langsung mencari pola.

Metode: Binarization (Pengubahan ke 0/1) Kita harus memecah atribut menjadi item baru.

Data Asli: {Jenis: Baju, Warna: Merah}

Data Biner: Item_Baju = 1, Item_Celana = 0, Item_Merah = 1, Item_Biru = 0.

Masalah: Jumlah item jadi meledak (banyak kolom baru). Solusi: Gunakan Apriori pada data hasil binarisasi ini, tapi hati-hati dengan atribut yang punya banyak nilai (misal: Kode Pos), karena bisa memakan memori.


**Handling Continuous Attributes (Menangani Angka/Kontinu)**

Bagaimana mencari pola pada atribut seperti Usia atau Gaji?Aturan: {Usia: 25} -> {Beli: Game} berbeda dengan {Usia: 25.5}.

Ada beberapa metode yang dibahas:
- Discretization (Pengelompokan/Binning)Mengubah angka menjadi rentang (interval).

Contoh: Usia 0-10 (Anak), 11-20 (Remaja), 21-60 (Dewasa).

Tantangan:Jika rentang terlalu lebar $\rightarrow$ Pola hilang (Information Loss).Jika rentang terlalu sempit $\rightarrow$ Support terlalu kecil, pola tidak ketemu.

- Statistics-based Methods
Menggunakan aturan statistik (seperti rata-rata atau deviasi standar) untuk menentukan batasan rentang secara otomatis, bukan manual.

- Min-Apriori
Metode khusus di mana support dihitung berdasarkan nilai minimum dari atribut kontinu tersebut (jarang dipakai dibanding Discretization).

**Handling Concept Hierarchies (Multilevel Association)**

Ini bagian yang sangat penting di ritel. Barang punya level (Taxonomy).

Level 1 (Umum): Komputer

Level 2 (Kategori): Laptop

Level 3 (Spesifik): Laptop Gaming ASUS ROG

Masalah: Kalau kita mining di Level 3, support-nya pasti kecil (jarang orang beli spesifik ASUS ROG dibanding beli "Laptop" saja). Pola penting bisa terlewat karena kena pruning di awal.

Metode:

- Mining pada Level Tertinggi: Cari pola {Komputer} -> {Printer}. Support pasti tinggi, tapi informasinya kurang detail.

- Mining pada Level Terendah: Cari pola {ASUS ROG} -> {Mouse Logitech G502}. Support rendah, banyak yang hilang.

- Multilevel Association Rules (Solusi Terbaik):

Gunakan support threshold yang berbeda.

Level atas (Umum) pakai Min Support tinggi.

Level bawah (Spesifik) pakai Min Support rendah.



Redundansi (Redundancy Filtering): Jika kita menemukan aturan:

{Susu} -> {Roti} (Support 10%, Confidence 80%)

{Susu Cokelat} -> {Roti} (Support 2%, Confidence 82%)

Aturan ke-2 mungkin adalah Redundant Rule karena dia cuma "anak" dari aturan pertama.


**Sequential Pattern Mining (Analisis Urutan Waktu)**


Apriori tidak peduli urutan (Beli A dan B itu sama dengan B dan A).

Sequence Mining: Urutan itu MUTLAK.
Pola: {Beli HP} $\rightarrow$ (minggu depan) {Beli Casing} $\rightarrow$ (bulan depan) {Ganti LCD}.

Kalau urutannya dibalik {Ganti LCD} -> {Beli HP}, itu aneh. Apriori tidak bisa menangkap ini.

Algoritma: GSP (Generalized Sequential Pattern), SPADE, PrefixSpan

**Subgraph/Graph Mining**

Mencari pola bukan di tabel, tapi di Jaringan (Graph).

Contoh: Analisis jejaring sosial (Siapa berteman dengan siapa membentuk pola segitiga?), atau struktur kimia (Senyawa obat kanker punya pola struktur cincin C-C-O).

**Infrequent & Negative Patterns**

Mencari "Anti-Pola".

Infrequent: Barang yang sangat jarang muncul (bisa jadi barang antik/mahal, atau anomali/fraud).

Negative: Aturan "Jika beli X, JANGAN tawarkan Y". (Produk yang saling mematikan).